In [1]:
import numpy as np
import pandas as pd
import os
import csv
import zipfile
import re
pd.set_option('display.max_columns', 500)

In [2]:
src_dir = os.getcwd()
if not os.path.exists(os.path.join(src_dir,"MySQL_Folders")):
    os.makedirs(os.path.join(src_dir,"MySQL_Folders"))
dst_dir = os.path.join(src_dir,'MySQL_Folders')
outputs_dir = os.path.join(os.getcwd(),'MySQL_Outputs')
output_zips = os.listdir(os.path.join(os.getcwd(),'MySQL_Outputs'))
dba_users_columns = ['Host', 'User', 'Password', 'Select_priv', 'Insert_priv', 'Update_priv', 'Delete_priv',
                     'Create_priv', 'Drop_priv', 'Reload_priv', 'Shutdown_priv', 'Process_priv', 'File_priv',
                     'Grant_priv', 'References_priv', 'Index_priv', 'Alter_priv', 'Show_db_priv', 'Super_priv',
                     'Create_tmp_table_priv', 'Lock_tables_priv', 'Execute_priv', 'Repl_slave_priv', 'Repl_client_priv',
                     'Create_view_priv', 'Show_view_priv', 'Create_routine_priv', 'Alter_routine_priv', 'Create_user_priv',
                     'Event_priv', 'Trigger_priv', 'Create_tablespace_priv', 'ssl_type', 'ssl_cipher', 'x509_issuer',
                     'x509_subject', 'max_questions', 'max_updates', 'max_connections', 'max_user_connections', 'plugin',
                     'authentication_string', 'password_expired', 'db']
dba_users = pd.DataFrame(columns=dba_users_columns)
grants = pd.DataFrame(columns=['values', 'db'])
nomina = pd.read_excel('nomina.xlsx', engine='openpyxl')
nomina['Nombre de usuario'] =  nomina['Nombre de usuario'].apply(lambda x: str(x).lower())

In [3]:
for o in enumerate(output_zips):
    #Create path to unzip and drop 
    newpath = os.path.join(dst_dir, output_zips[o[0]].replace(".zip","")) 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    with zipfile.ZipFile(os.path.join(outputs_dir,output_zips[o[0]]), 'r') as zip_ref:
        zip_ref.extractall(newpath)
    
    #list files in every unzipped folder, files 0 corresponds to dba_user file and files 1 to grants file.
    files = os.listdir(newpath)
    dba_users_f = files[0]
    grants_f = files[1]
    dba_users = dba_users.append(pd.read_csv(os.path.join(newpath, dba_users_f)), sort=False, ignore_index=True)
    dba_users['db'] = dba_users['db'].fillna(output_zips[o[0]].replace(".zip",""))
    grants = grants.append(pd.read_csv(os.path.join(newpath, grants_f), header=None, names=['values']), sort=False, ignore_index=True)
    grants['db'] = grants['db'].fillna(output_zips[o[0]].replace(".zip",""))
    
    #Boolean mask to filter relevant privileges/grants
    mask_relevant_permissions = (dba_users['password_expired'] == 'N') & ((dba_users['Insert_priv'] == 'Y') | \
                                (dba_users['Update_priv'] == 'Y') | (dba_users['Delete_priv'] == 'Y') | \
                                (dba_users['Create_priv'] == 'Y') | (dba_users['Drop_priv'] == 'Y') | \
                                (dba_users['Grant_priv'] == 'Y') | (dba_users['Alter_priv'] == 'Y') | \
                                (dba_users['Super_priv'] == 'Y') | (dba_users['Create_user_priv'] == 'Y'))
    relevant_grants = ['INSERT', 'UPDATE', 'DELETE', 'CREATE', 'CREATE',
                       'DROP', 'WITH GRANT OPTION', 'ALTER', 'SUPER', 'CREATE USER', 'ALL PRIVILEGES']
    
    #Overwrite dataframe with filters applied
    dba_users = dba_users[mask_relevant_permissions]
    
    #Regex to get relevant grants defined previosly
    esc_lst = [re.escape(s) for s in relevant_grants]
    pattern = '|'.join(esc_lst)
    grants = grants[grants['values'].str.contains(pattern, case=False)]
    
    #Data cleansing
    grants['user'] = grants['values'].apply(lambda x: re.findall("(?<= TO ')[a-zA-Z0-9-_]+|(?<= TO `)[a-zA-Z0-9-_]+",x))
    grants['user'] = grants['user'].astype(str).apply(lambda x: x.replace("[",""))
    grants['user'] = grants['user'].apply(lambda x: x.replace("]",""))
    grants['user'] = grants['user'].apply(lambda x: x.replace("'",""))
    
    print('Loop generated for ', o[1], ' file: current shape: ', '\n', 'dba_users: ', dba_users.shape, '\n', 'grants: ', grants.shape)

Loop generated for  aml02.zip  file: current shape:  
 dba_users:  (22, 53) 
 grants:  (33, 3)
Loop generated for  aml03.zip  file: current shape:  
 dba_users:  (44, 53) 
 grants:  (66, 3)
Loop generated for  aml04.zip  file: current shape:  
 dba_users:  (66, 53) 
 grants:  (99, 3)
Loop generated for  aml05.zip  file: current shape:  
 dba_users:  (88, 53) 
 grants:  (132, 3)
Loop generated for  aml06.zip  file: current shape:  
 dba_users:  (110, 53) 
 grants:  (165, 3)
Loop generated for  amlrules.zip  file: current shape:  
 dba_users:  (132, 53) 
 grants:  (202, 3)


In [4]:
dba_users = dba_users[['db', 'User','password_expired', 'Insert_priv', 'Update_priv', 
                       'Delete_priv', 'Create_priv', 'Drop_priv', 'Grant_priv',
                       'Alter_priv', 'Super_priv', 'Create_user_priv']]
dba_users = dba_users.merge(nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='User', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
grants = grants.merge(nomina[['Nombre de usuario', 'Departamento']], how='left', left_on='user', right_on='Nombre de usuario').drop('Nombre de usuario', axis=1)
common_users = ['ab_schemas', 'addslave', 'alter_qa', 'consultas_exc', 'ggs11', 'mysql', 'root', 'sentinel']
own_default_users = ('_ADMIN', '_WPROD', '_MIGRA')
c1 = dba_users['User'].str.contains('|'.join(common_users))
c2 = dba_users['User'].str.endswith(own_default_users)
c3 = grants['user'].str.contains('|'.join(common_users))
c4 = grants['user'].str.endswith(own_default_users)
dba_users['Departamento'][dba_users['Departamento'].isnull()] = np.select([c1,c2], 
                                                                          [dba_users['Departamento'].fillna('Usuario creado por API comun a todas las bases'),
                                                                           dba_users['Departamento'].fillna('Usuario creado por API comun a todas las bases')])
grants['Departamento'][grants['Departamento'].isnull()] = np.select([c3,c4], 
                                                                          [grants['Departamento'].fillna('Usuario creado por API comun a todas las bases'),
                                                                           grants['Departamento'].fillna('Usuario creado por API comun a todas las bases')])

In [5]:
ad_users = pd.read_excel('AD.xlsx',engine='openpyxl')
ad_users = ad_users[['Name', 'Disabled']]

In [6]:
dba_users = dba_users.merge(ad_users, how='left', left_on='User', right_on='Name').drop('Name', axis=1)
grants = grants.merge(ad_users, how='left', left_on='user', right_on='Name').drop('Name', axis=1)
dba_users['Disabled'][dba_users['Disabled'].isnull()] = np.select([c1,c2], 
                                                                  [dba_users['Disabled'].fillna('N/A'),
                                                                   dba_users['Disabled'].fillna('N/A')])
grants['Disabled'][grants['Disabled'].isnull()] = np.select([c3,c4],
                                                            [grants['Disabled'].fillna('N/A'),
                                                             grants['Disabled'].fillna('N/A')])

In [7]:
dba_users.to_csv('total_dba_users.csv')
grants.to_csv('total_grants.csv')